<br>

# Homework 5 - Exploring Wikipedia's hyperlinks graph!

#### Samin Hamidi, Katerina Zavadskaya

In [26]:
#Importing libraries we need
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from collections import deque
import random
import math
import time
import networkx 
import collections
from tqdm import tqdm
import csv

# RQ1

In [2]:
# reading reduced-graph file as a data-frame
df = pd.read_csv('wiki-topcats-reduced.txt', sep=r'\t', header=None,  names=['Source','Destination'])

In [3]:
df.head()

,Source,Destination
0,52,401135
1,52,1069112
2,52,1163551
3,62,12162
4,62,167659


__Number of Edges__

In [4]:
num_of_edges = df.shape[0]
print('Number of Edges:', num_of_edges)

Number of Edges: 2645247


__Number of Nodes__

In [5]:
nodes = set(df['Source'].values).union(set(df['Destination'].values))
num_of_nodes = len(nodes)
print('Number of Nodes:', num_of_nodes)

Number of Nodes: 461193


__Checking to see if our graph is directed?__<br>
<br>
we create the graph once for source nodes as the keys and once for destination nodes as our keys. If intersection set between a similar key in two dictionaries is not empty, we have a proof to show our graph is directed. 

In [6]:
# dictionary with source nodes as the key
out_Link = df.groupby(['Source'])['Destination'].apply(list).to_dict()

In [7]:
# dictionary with destination nodes as the key
in_Link = df.groupby(['Destination'])['Source'].apply(list).to_dict()

In [8]:
# checking how many similar keys have non empty intersection sets?
intersect_ = {}
intersect = 0
for item in out_Link.keys(  ):
    if item in in_Link.keys():
        intersect = set(out_Link[item]).intersection(set(in_Link[item]))
        if len(intersect)>0:   
            intersect_[item] = set(out_Link[item]).intersection(set(in_Link[item]))
print(len(intersect_))

249369


__We have enough proof to show that our graph is Directed!__<br><br>

__The average node degree__

In [9]:
average_degree = num_of_edges//num_of_nodes
print('average node degree:', average_degree)

average node degree: 5


__Checking if our graph is a dense graph?__

In [10]:
D = num_of_edges/(num_of_nodes*(num_of_nodes-1))
print('graph density:', D)
# if D is close to 1 it means that graph is dense
# formula from wiki for directed graphs
# D = |E| / |V|*(|V|-1)
# so our graph is not dense (close to 0)

graph density: 1.2436602635647606e-05


If D is close to 1 it means that graph is dense, so our graph is not dense (close to 0).

<br>

# Categories dataset and cleaning the data <br>


__Categories data set__

In [11]:
# We use a dictionary for saving categories data. Keys are categories' titles and values are lists of each category's articles(nodes)
categories = {}
with open('wiki-topcats-categories.txt') as file:
    for list_ in file:
        if 3500<len(list_.split()[1:])<20000: #we choose only those categories with number of nodes bigger that 3500 and smaller 20000
            #this particular choice helps us to reduce the runing time and actully to be able to run the code
            categories[list_.split()[0][9:-1]] = list(map(int,list_.split()[1:])) #{'1234': [123, 1234, 88,...], ...}

In [12]:
categories.keys()

dict_keys(['English_footballers', 'The_Football_League_players', 'Association_football_forwards', 'Association_football_goalkeepers', 'Association_football_midfielders', 'Association_football_defenders', 'Year_of_birth_unknown', 'Harvard_University_alumni', 'Major_League_Baseball_pitchers', 'Members_of_the_United_Kingdom_Parliament_for_English_constituencies', 'Indian_films', 'Year_of_death_missing', 'English_cricketers', 'Rivers_of_Romania', 'Main_Belt_asteroids', 'Asteroids_named_for_people', 'English-language_albums', 'English_television_actors', 'British_films', 'American_films', 'Fellows_of_the_Royal_Society', 'People_from_New_York_City', 'American_Jews', 'American_television_actors', 'American_film_actors', 'Debut_albums', 'Black-and-white_films', 'Year_of_birth_missing', 'Place_of_birth_missing_(living_people)', 'Article_Feedback_Pilot', 'American_military_personnel_of_World_War_II', 'Windows_games'])

In [13]:
# Number of categories we have
len(categories.keys())

32

As our graph is a reduced version, some of the nodes in categories might not exist in the reduced graph so we need to consider this and clean the categories dataframe and categories dictionary. 

In [14]:
# We want only those nodes that are in the reduced graph.
graph_categories = {}
for key in categories.keys():
    article_set = set(categories.get(key)).intersection(nodes)
    graph_categories.update({key:article_set})

In [15]:
graph_categories_df = pd.DataFrame()
graph_categories_df['Categories'] = list(graph_categories.keys())
graph_categories_df['Articles'] = list(graph_categories.values())
art_len = [len(elem) for elem in list(graph_categories.values())]
graph_categories_df['Count'] = art_len

In [16]:
graph_categories_df

,Categories,Articles,Count
0,English_footballers,"{622642, 622644, 622647, 622649, 557114, 62266...",7538
1,The_Football_League_players,"{1671189, 622644, 622647, 622649, 557114, 1245...",7814
2,Association_football_forwards,"{1671189, 884758, 884785, 1671225, 884795, 884...",5097
3,Association_football_goalkeepers,"{737292, 737293, 1671183, 73743, 106515, 10651...",3737
4,Association_football_midfielders,"{1671168, 884755, 884774, 622637, 1671214, 167...",5827
5,Association_football_defenders,"{81920, 1359875, 1359879, 802827, 81936, 10651...",4588
6,Year_of_birth_unknown,"{958476, 958480, 172050, 368658, 368661, 21301...",2536
7,Harvard_University_alumni,"{1638406, 1638409, 1245204, 294956, 1048629, 6...",5549
8,Major_League_Baseball_pitchers,"{1671480, 1278327, 1278328, 328097, 1278406, 3...",5192
9,Members_of_the_United_Kingdom_Parliament_for_E...,"{1048670, 1147049, 1147262, 1147295, 1507751, ...",6491


In [17]:
# after taking the intersection of each categories' value list with unique nodes in our reduced graph, we again check to 
# select those categories with higer value length of 3500
graph_categories_df = graph_categories_df[graph_categories_df['Count']>3500].reset_index(drop=True)

In [18]:
graph_categories_df

,Categories,Articles,Count
0,English_footballers,"{622642, 622644, 622647, 622649, 557114, 62266...",7538
1,The_Football_League_players,"{1671189, 622644, 622647, 622649, 557114, 1245...",7814
2,Association_football_forwards,"{1671189, 884758, 884785, 1671225, 884795, 884...",5097
3,Association_football_goalkeepers,"{737292, 737293, 1671183, 73743, 106515, 10651...",3737
4,Association_football_midfielders,"{1671168, 884755, 884774, 622637, 1671214, 167...",5827
5,Association_football_defenders,"{81920, 1359875, 1359879, 802827, 81936, 10651...",4588
6,Harvard_University_alumni,"{1638406, 1638409, 1245204, 294956, 1048629, 6...",5549
7,Major_League_Baseball_pitchers,"{1671480, 1278327, 1278328, 328097, 1278406, 3...",5192
8,Members_of_the_United_Kingdom_Parliament_for_E...,"{1048670, 1147049, 1147262, 1147295, 1507751, ...",6491
9,Indian_films,"{589824, 589825, 589826, 589827, 589828, 58982...",5568


__Doing the same cleaning for the dictionary__

In [19]:
# for having in our dictionary only categories with more than 3500 articles
for elem in list(graph_categories.keys()):
    if elem not in graph_categories_df['Categories'].tolist():
        del graph_categories[elem]

In [20]:
len(graph_categories)

26

__all the noeds after cleaning the categories__

In [21]:
len(sorted({x for v in graph_categories.values() for x in v}))

139433

<br>

# RQ2 <br>
# a. Find the Shortest Path

There are options for choosing an algorithm that calculates the shortest path among each two nodes. We pick BFS(Breadth First Search). This algorithm suits our task the best as our graph is not weighted and BFS also works for directed graphs. <br> 
Given a graph $G = (V,E)$ and a distinguished source vertex $s$, breadth-ﬁrst search systematically explores the edges of $G$ to “discover” every vertex that is reachable from $ S $, It computes the distance (smallest number of edges) from $s$ to each reachable vertex. It also produces a “breadth-ﬁrst tree” with root $s$ that contains all reachable vertices. For any vertex $v$ reachable from $s$, the simple path in the breadth-ﬁrst tree from $s$ to  corresponds to a “shortest path” from $s$ to $v$ in $G$, that is, a path containing the smallest number of edges. The algorithm works on both directed and undirected graphs. 

In [22]:
#Input Category
C0_name = 'English_footballers'
C0 = list(graph_categories[C0_name])[0:1000]# we choose the first 1000 nodes in input category 'English_footballers'
len(C0)

1000

In [23]:
def BFS_with_saving_path(graph, start):
    visited = {start}
    queue = deque([(start, [])])
    sav_path = {}
    
    while queue:
        current, path = queue.popleft()
        try:
            for neighbor in graph[current]:
                if neighbor not in visited:
                    queue.append((neighbor, path + [current]))
                    visited.add(neighbor)
                    sav_path[neighbor] = current
        except KeyError:
            pass
    return sav_path

In [24]:
def ShortestPath(graph, start, goal, tree):
    path = []
    if start == goal:
        return(len(path))
    elif goal not in tree:
        return(math.inf)
    else:
        current = goal
        while current != start:
            path.append(tree[current])
            current = tree[current]
        return(len(path))

__new-Katerina__

In [ ]:
path_Ci_Cj = []
i = 1
for category in tqdm(list(graph_categories.values())[1:]):
    for start in C0:
        tree = BFS_with_saving_path(out_Link, start)
        for end in category:
            path = ShortestPath(out_Link,start,end,tree)
            path_Ci_Cj.append(path)  # [5,2,4,inf,3]
            
    w = csv.writer(open('C'+str(i)+'.csv', 'w'))
    for item in path_Ci_Cj:
        w.writerow([item])
        
    i = i + 1
    path_Ci_Cj = []


 48%|████████████████████████████████████▉                                        | 12/25 [2:54:55<3:00:10, 831.60s/it]

__new-Samin__

In [35]:
def cal_(C0, key):
    Ci = graph_categories[key]#to calculate shortest paths for pairs in (C0, Ci)
    path_C0_Ci = []
    for start in tqdm(C0):#source node in C0
        tree = BFS_with_saving_path(out_Link, start)#gives us a tree from start in C0 and all the other nodes in the graph
        for end in Ci:#destination node in category Ci
            shortest_path_value = ShortestPath(out_Link, start, end, tree)
            path_C0_Ci.append(shortest_path_value) # [5,2,4,3]   
    return(path_C0_Ci)

In [ ]:
ranks = []
Ci_to_Cj_sp_values = []
for key in list(graph_categories.keys())[0:2]:
    if key != C0_name:
        Ci_to_Cj_sp_values = cal_(C0, key) # a list of values each for the shortest paths between nodes in C0 and Ci
    #find the median
    ranks.append(tuple((key, np.median(Ci_to_Cj_sp_values)))) 

# b. Rank all the nodes in each category

In [26]:
# creating the initial graph as a list of tuples using NetworkX 
initial_gr = networkx.DiGraph(out_Link)

In [27]:
#creating an induced graph from input category C0
C0 =  list(graph_categories.values())[0] #category "English_footballers"
C0_sub = initial_gr.subgraph(C0)#induced subgraph fron C0
in_degree_dict = C0_sub.in_degree(C0)#gives us a dictionary of all the nodes with in-coming edges as keys and their degree
# we need to sort based on the degree of each node
sorted_ = []
sorted_.append(sorted(in_degree_dict.items(), key=lambda x: x[1]))# creating a list of tuples out of the dictionary, sorted by the second item of each tuple which is node in_degree

In [28]:
for category in tqdm(list(graph_categories.values())[1:]): #from the second category(C1) onward in category's dictionary 
    empty_ = []
    Ci = category 
    
    Ci = Ci.difference(C0)# we don't consider nodes shared among Ci and C0 as those already have their scores
    
    induced_gr = initial_gr.subgraph(set(C0).union(set(Ci)))#creating an induced graph of the union of Ci and C0
    
    for i, node in enumerate(Ci):#calculating the in_degree score for all the nodes in Ci (Ci = Ci.difference(C0))
        score = 0
        if len(induced_gr.in_edges(node)) == 0: #if  list of in-coming edges for node is empty then score = 0
            score = 0
    
        else:
            for item in induced_gr.in_edges(node): # we check for every item in the form of (source, destination) in in-coming edges list 
                if item[0] in Ci: #if source is in Ci, score+=1
                    score += 1
                if item[0] in list(in_degree_dict.keys()):# if not in Ci, then that edge has the weight equal to source node's score
                    score += in_degree_dict[item[0]]
        
        empty_.append((node,score))
    

    empty_ = sorted(empty_, key=lambda x: x[1])
    sorted_.append(empty_) #a list of lists. Each list comprised by tuples 
    
    C0 = induced_gr.nodes()

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [01:40<00:00,  4.14s/it]


In [ ]:
# here put code for saving sorted_ to a file
w = csv.writer(open("R2_sorted_nodes.csv", "w"))
for item in sorted_:
    w.writerow([item])